# Testing Agent Components

I used this notebook to test different tools and components.  

In [ ]:
from agents import manager_agent
manager_agent.visualize()

In [ ]:
from PIL import Image

# Path to image file
picture_path = './bird_image.png'

# List to hold images
images = []

# Open image from path
image = Image.open(picture_path)

# Ensure the image is loaded properly and append it to the list of images
if image:
    images.append(image.convert('RGB'))

#manager_agent.run("How many bird species are in this image?", images=images)
manager_agent.run("In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?")

In [ ]:
manager_agent.run('How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.')

In [ ]:
manager_agent.run("How many r's are in the word Strawberry?")


In [ ]:
from tools.tools import youtube_frames_to_images
results = youtube_frames_to_images('https://youtube.com/watch?v=L1vXCYZAYYM')

In [ ]:
from tools.tools import youtube_transcribe
results = youtube_transcribe('https://www.youtube.com/watch?v=1htKBjuUWec')
print(results)

In [ ]:
from PIL import Image

# Path to image file
picture_path = './bird_image.png'

# List to hold images
images = []

# Open image from path
image = Image.open(picture_path)

# Ensure the image is loaded properly and append it to the list of images
if image:
    images.append(image.convert('RGB'))

from tools.tools import use_vision_model

query = "How many bird species are in this picture?"

tool_input = {
    "question": query,
    "images": images
}

use_vision_model(**tool_input)